# Amharic Sentiment Analysis - Getting Started

This notebook demonstrates how to use the Amharic Sentiment Analysis package for:
1. Loading and preprocessing data
2. Training different model architectures
3. Evaluating model performance
4. Making predictions on new text

## Setup

First, let's install the package and import the necessary modules.

In [ ]:
# Add project root to path (for development)
import sys
sys.path.insert(0, '..')

# Import modules
from amharic_sentiment.data.dataset import AmharicSentimentDataset
from amharic_sentiment.preprocessing.pipeline import PreprocessingPipeline
from amharic_sentiment.models import CNN, BiLSTM, GRU, CNNBiLSTM
from amharic_sentiment.training.trainer import Trainer
from amharic_sentiment.evaluation.metrics import evaluate_model, get_classification_report
from amharic_sentiment.evaluation.visualize import (
    plot_training_history, 
    plot_confusion_matrix,
    plot_metrics_comparison
)

import warnings
warnings.filterwarnings('ignore')

print("Modules imported successfully!")

## 1. Data Loading and Preprocessing

Let's load the Amharic sentiment dataset and explore its contents.

In [ ]:
# Initialize dataset
dataset = AmharicSentimentDataset(
    max_words=15000,  # Maximum vocabulary size
    max_len=20        # Maximum sequence length
)

# Load data from files
dataset.load_from_files(
    positive_file='../dataset/postive comment.txt',
    negative_file='../dataset/negative comment.txt'
)

# Get dataset statistics
stats = dataset.get_stats()
print("Dataset Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value}")

In [ ]:
# View some sample texts
print("Sample positive texts:")
for i, (text, label) in enumerate(zip(dataset.texts[:3], dataset.labels[:3])):
    print(f"  {i+1}. {text[:60]}...")

print("\nSample negative texts:")
negative_indices = [i for i, l in enumerate(dataset.labels) if l == 0][:3]
for idx in negative_indices:
    print(f"  - {dataset.texts[idx][:60]}...")

### Text Preprocessing

The preprocessing pipeline handles:
- URL removal
- Amharic punctuation cleaning
- Character normalization (handling Unicode variants)
- Labialized character normalization

In [ ]:
# Test preprocessing pipeline
pipeline = PreprocessingPipeline()

sample_text = "ሰላም!! https://example.com ይህ ጥሩ ነው 123"
processed = pipeline.process(sample_text)

print(f"Original: {sample_text}")
print(f"Processed: {processed}")

## 2. Training Models

Let's train the CNN-BiLSTM model (best performing architecture).

In [ ]:
# Create trainer for CNN-BiLSTM
trainer = Trainer(
    model_type='cnn_bilstm',
    dataset=dataset,
    output_dir='../experiments',
    experiment_name='cnn_bilstm_demo',
    model_params={
        'embedding_dim': 32,
        'filters': 64,
        'lstm_units': 64,
        'dropout_rate_conv': 0.2,
        'dropout_rate_lstm': 0.3
    }
)

print("Trainer initialized!")

In [ ]:
# Prepare data splits
data_splits = trainer.prepare_data(
    test_size=0.1,
    val_size=0.1,
    random_state=42
)

print(f"Training samples: {len(data_splits['y_train'])}")
print(f"Validation samples: {len(data_splits['y_val'])}")
print(f"Test samples: {len(data_splits['y_test'])}")

In [ ]:
# Train the model
history = trainer.train(
    epochs=5,  # Use more epochs for better results
    batch_size=32,
    learning_rate=0.001,
    verbose=1
)

## 3. Model Evaluation

Let's evaluate the trained model and visualize the results.

In [ ]:
# Evaluate on test set
results = trainer.evaluate(verbose=1)

print("\nTest Results:")
print(f"  Accuracy:  {results['accuracy']:.4f}")
print(f"  Precision: {results['precision']:.4f}")
print(f"  Recall:    {results['recall']:.4f}")
print(f"  F1 Score:  {results['f1_score']:.4f}")
print(f"  ROC AUC:   {results['roc_auc']:.4f}")

In [ ]:
# Plot training history
plot_training_history(history, metrics=['loss', 'accuracy'])

In [ ]:
# Plot confusion matrix
plot_confusion_matrix(
    results['confusion_matrix'],
    labels=['Negative', 'Positive']
)

## 4. Making Predictions

Let's use the trained model to predict sentiment on new text.

In [ ]:
# Sample texts for prediction
test_texts = [
    "ጥሩ ስራ ነው ተባረኩ",           # Positive: Good work, be blessed
    "በጣም መጥፎ ውሳኔ ነው",           # Negative: Very bad decision
    "እግዚአብሔር ይስጥልን",          # Positive: May God give us
    "ሀገር አፍራሽ",                  # Negative: Country destroyer
]

# Preprocess and tokenize
processed_texts = [pipeline.process(text) for text in test_texts]
sequences = dataset.tokenizer.texts_to_sequences(processed_texts)

from tensorflow.keras.preprocessing.sequence import pad_sequences
padded = pad_sequences(sequences, maxlen=20, padding='post')

# Predict
predictions = trainer.model.predict_proba(padded)

print("Predictions:")
print("-" * 60)
for text, prob in zip(test_texts, predictions):
    sentiment = "Positive" if prob > 0.5 else "Negative"
    confidence = prob if prob > 0.5 else 1 - prob
    print(f"Text: {text}")
    print(f"  Sentiment: {sentiment} (confidence: {confidence:.2%})")
    print()

## 5. Comparing Different Models

Let's train and compare different model architectures.

In [ ]:
# Train multiple models for comparison (simplified for demo)
model_types = ['cnn', 'bilstm', 'gru', 'cnn_bilstm']
comparison_results = {}

for model_type in model_types:
    print(f"\nTraining {model_type.upper()}...")
    
    # Create new dataset instance
    ds = AmharicSentimentDataset(max_words=15000, max_len=20)
    ds.load_from_files(
        '../dataset/postive comment.txt',
        '../dataset/negative comment.txt'
    )
    
    # Create trainer
    t = Trainer(
        model_type=model_type,
        dataset=ds,
        output_dir='../experiments/comparison',
        experiment_name=f'{model_type}_comparison'
    )
    
    # Train (fewer epochs for demo)
    t.train(epochs=3, batch_size=64, verbose=0)
    
    # Evaluate
    results = t.evaluate(verbose=0)
    comparison_results[model_type] = {
        'accuracy': results['accuracy'],
        'precision': results['precision'],
        'recall': results['recall'],
        'f1_score': results['f1_score']
    }
    print(f"  Accuracy: {results['accuracy']:.4f}")

print("\nTraining complete!")

In [ ]:
# Visualize comparison
plot_metrics_comparison(comparison_results)

## 6. Saving and Loading Models

Save the trained model for later use.

In [ ]:
# Save the model
model_path = trainer.save_model()
print(f"Model saved to: {model_path}")

# Save the tokenizer
tokenizer_path = str(trainer.output_dir / 'tokenizer.pkl')
dataset.save_tokenizer(tokenizer_path)
print(f"Tokenizer saved to: {tokenizer_path}")

In [ ]:
# Load the model for inference
from tensorflow.keras.models import load_model
import pickle

# Load model
loaded_model = load_model(model_path)

# Load tokenizer
with open(tokenizer_path, 'rb') as f:
    loaded_tokenizer = pickle.load(f)

print("Model and tokenizer loaded successfully!")

## Summary

In this notebook, we demonstrated:

1. **Data Loading**: Loading Amharic text data from files
2. **Preprocessing**: Cleaning and normalizing Amharic text
3. **Model Training**: Training CNN-BiLSTM and other architectures
4. **Evaluation**: Measuring accuracy, precision, recall, F1 score
5. **Prediction**: Classifying new text samples
6. **Model Comparison**: Comparing different architectures
7. **Model Persistence**: Saving and loading trained models

For more advanced usage, see:
- `configs/` for configuration options
- `scripts/train.py` for command-line training
- API documentation in the source code